In [1]:
!pip install bentoml 

In [ ]:
%%writefile lstm_classifier.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
# from bentoml.frameworks.sklearn import SklearnModelArtifact
# from bentoml.adapters import JsonInput, JsonOutput
from bentoml.frameworks.pytorch import PytorchModelArtifact
from bentoml.service.artifacts.pickle import PickleArtifact

@env(infer_pip_packages=True)
@artifacts([PytorchModelArtifact('model')])
class LstmClassifier(BentoService):
    """
    이 예제는 전처리가 필요없는 간단한 데이터와 간단한 모델이므로 코드가 간결합니다.
    """

    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        """
        모델의 input에 맞게 ([0.0,0.0,0.0,0.0]) HTTP 요청을 Post로 받았을시 동작하는 코드입니다.
        요청은 http://HOST:PORT/predict로 보냅니다.
        """
        return self.artifacts.model.predict(df)

In [6]:
import torch.nn as nn
import os.path as p

# load model
model = nn.LSTM(64,64,2,batch_first=True)
model_path = p.join('/opt/ml/code/models', 'lstm.pt')
model.load_state_dict(torch.load(model_path),strict=False)
model.eval()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [3]:
from lstm_classifier import LstmClassifier

# 인스턴스화를 해준 다음
lstm_classifier_service = LstmClassifier()

# 이미 훈련한 모델과 함께 패키징을 합니다.
lstm_classifier_service.pack('model', model)

# 마지막으로 서비스 자체를 저장합니다.
saved_path = lstm_classifier_service.save()

CUDA: False -> cpu
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
# 데이터 전처리, 배치 작업까지
# 모델 선언, 파라미터 넘기기
# train eval loop
